<a href="https://colab.research.google.com/github/HodaMemar/Patient-Similarity-through-Representation/blob/main/Create_Gold_Standard.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In this section, a golden standard is produced to evaluate the quality of embedding vectors. The source of this gold standard is the registered final diagnoses for patients. In the structure of this golden standard, the method presented in the article https://doi.org/10.1016/j.artmed.2020.101900  is used. In this method, the order of recorded final diagnoses is used for weighting.

In [ ]:
import pandas as pd
gold=pd.read_csv('...\diagnoses.csv')
gold

In [ ]:
import numpy as np
Weights = {}
for i in range(1,50):
    Weights[i] = 2 / (np.exp(i-1) + 1)

ls_W=[]
for i in range(len(gold)):
    seq =gold.loc[i].SEQ_NUM
    ls_W.append(Weights[seq])
gold['Weight']=pd.DataFrame(ls_W)  

In [ ]:
def simIndex(a,b):
    x={}
    x=a
    y={}
    y=b
    #print(x,y)
    commonhadm = list(set(x.keys()).intersection(set(y.keys())))
    
    if (len(commonhadm) < 1):
            return 0.0
    aw,bw = 0.0,0.0
    for i in x:
        aw = aw +x[i] 
    for j in y:
        bw = bw +y[j]
    sum = 0
    for c in commonhadm:
            w1 = x[c]
            
            w2 = y[c]
            #print(w1,w2)
            sum = sum + min(w1, w2)
    similarityIndex = sum / np.mean([aw, bw])
    return round(similarityIndex, 4)


In [ ]:
import time
ls_residual=[]
 
i=0
for key_i in list_Subject_id :
    df_a=gold[gold['HADM_ID']==key_i][['ICD9_CODE','Weight']]
    a = dict(zip(df_a.ICD9_CODE, df_a.Weight))
    start = time.time()  
    
    i=0
    for key_j in list_Subject_id:
            try:
                df_b=gold[gold['HADM_ID']==key_j][['ICD9_CODE','Weight']]
                b = dict(zip(df_b.ICD9_CODE, df_b.Weight))
                
                i=i+1
                score1.append([key_i,key_j,simIndex(a,b)])
                
            except:
                print("An exception occurred")
           
             
    stop = time.time()
    duration = stop-start
    print(key_i,i,duration,'----------')  

In [ ]:
gold_df=pd.DataFrame(score1,columns=['p1','p2','SimIndex'])
gold_df.to_csv('...\gold_standard.csv', index = False)